In [ ]:
import torch
import os
from torch.utils.data import DataLoader
from torchmetrics import JaccardIndex
import segmentation_models_pytorch as smp

# --- 1. CẤU HÌNH ---
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# Trỏ vào folder TRAIN (Nơi duy nhất có Mask để chấm điểm)
DATA_DIR = '/kaggle/input/deepglobe-land-cover-classification-dataset/train' 

# --- 2. TẠO DATASET CHẤM ĐIỂM ---
# (Dùng lại class CloudDataset bạn đã định nghĩa trước đó)
# Lưu ý: Pass preprocessing_fn vào để chuẩn hóa ảnh giống lúc train
val_dataset = CloudDataset(
    images_dir=DATA_DIR,
    masks_dir=DATA_DIR, 
    preprocessing=preprocessing_fn, # Đảm bảo bạn đã khai báo preprocessing_fn ở trên
    has_masks=True
)

# --- MẸO: CHỈ LẤY 50 ẢNH ĐỂ CHẤM CHO NHANH ---
# Lấy danh sách tất cả ID
all_ids = [os.path.splitext(f)[0].replace('_sat', '') for f in os.listdir(DATA_DIR) if f.endswith('_sat.jpg')]

# Chỉ lấy 50 ảnh cuối cùng (Giả sử model chưa học kỹ phần này hoặc coi như tập Val)
val_ids = all_ids[400:600] 

# Gán đè danh sách ID này vào dataset
val_dataset.ids = val_ids
val_dataset.images_fps = [os.path.join(DATA_DIR, i + '_sat.jpg') for i in val_ids]
val_dataset.masks_fps = [os.path.join(DATA_DIR, i + '_mask.png') for i in val_ids]

print(f"Dataset đánh giá đã sẵn sàng với {len(val_dataset)} ảnh.")

# --- 3. LOADER & METRIC ---
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)

loss = smp.losses.DiceLoss(mode='binary')
loss.__name__ = 'dice_loss'

iou = JaccardIndex(task="binary", threshold=0.5).to(DEVICE)
iou.__name__ = 'iou_score'

# --- 4. CHẠY CHẤM ĐIỂM (VALID EPOCH) ---
# Load model (nếu chưa load)
# model.load_state_dict(torch.load('/kaggle/working/best_model.pth')) 
model.to(DEVICE)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=[iou], 
    device=DEVICE,
    verbose=True,
)

print("\nĐANG TÍNH ĐIỂM IoU TRÊN 200 ẢNH MẪU...")
logs = valid_epoch.run(val_loader)

print("\n" + "="*30)
print(f"KẾT QUẢ IoU: {logs['iou_score']:.4f}")
print("="*30)